In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
import matplotlib
import tensorly as tl
import json
import time
import random
from tensorly.decomposition import non_negative_tucker
import collections
from pandas import Series
from urllib.request import urlopen,quote
import networkx as nx
from networkx.algorithms import bipartite
import threading
import datetime

In [2]:
class MyThread(threading.Thread):
    def __init__(self,func,args,name=''):
        threading.Thread.__init__(self)
        self.func = func
        self.name = name
        self.args = args

    def run(self):
        #print('开始执行',self.name,' 在：',ctime())
        self.res = self.func(*self.args)
        #print(self.name,'结束于：',ctime())

    def getResult(self):
        return self.res

def myfun(df_q_kakou,df_jiedian,jiedian_id_ord,des_kid_ord,q_kakou_time_ord):
    #5对应jiedian_id；7对应des_kid；9对应q_kakou_time
    edge=[]
    for i in range(len(df_q_kakou)):
        #print(i)
        df_h_jiedian=df_jiedian[(df_jiedian['start_kid']==df_q_kakou.iloc[i,des_kid_ord]) & (df_jiedian['start_time']>df_q_kakou.iloc[i,q_kakou_time_ord])]   
        df_h_jiedian_index=list(df_h_jiedian.index)
        for h_index in df_h_jiedian_index:
            edge.append([str(df_q_kakou.iloc[i,jiedian_id_ord]),str(h_index)+'^'])
    return edge

In [3]:
#根据节点df_jiedian和卡口间时间df_kakou_time，同时指定线程数renwu_num，存储边到save_path
def save_edge(df_jiedian,df_kakou_time,renwu_num,save_path):
    ######前节点与卡口做连接，可得该前节点可以通过的卡口，
    df_q_kakou=pd.merge(df_jiedian,df_kakou_time,left_on='end_kid',right_on='origin_kid',how='inner')
    df_q_kakou['q_kakou_time']=df_q_kakou['time']+df_q_kakou['end_time']
    #print(len(df_jiedian))
    #print(len(df_kakou_time))
    #print(len(df_q_kakou))
    ######获取列的索引
    df_q_kakou_columns=df_q_kakou.columns.values
    for c in range(len(df_q_kakou_columns)):
        if df_q_kakou_columns[c]=='des_kid':
            des_kid_ord=c
        if df_q_kakou_columns[c]=='q_kakou_time':
            q_kakou_time_ord=c
        if df_q_kakou_columns[c]=='jiedian_id':
            jiedian_id_ord=c
    ######使用多线程计算可连接的边
    renwu_num=renwu_num#开启线程数
    length=len(df_q_kakou)//renwu_num
    #print('行数:',len(df_q_kakou))
    df_q_kakou_list=[]
    for i in range(renwu_num-1):
        df_q_kakou_list.append(df_q_kakou[i*length:(i+1)*length])
    df_q_kakou_list.append(df_q_kakou[(renwu_num-1)*length:])
    #存储可连接的边
    myfun_l=[]
    for i in range(renwu_num):
        t = MyThread(myfun,args=(df_q_kakou_list[i],df_jiedian,jiedian_id_ord,des_kid_ord,q_kakou_time_ord))
        t.start()
        myfun_l.append(t)
    file=open(save_path,'w')#打开一个文件只用于写入。如果该文件已存在则打开文件，并从开头开始编辑，即原有内容会被删除。如果该文件不存在，创建新文件。
    for t in myfun_l:
        t.join()  # 一定要join，不然主线程比子线程跑的快，会拿不到结果
        edges=t.getResult()
        for edge in edges:
            file.write(edge[0]+' '+edge[1]+'\n')
    file.close()
    

In [4]:
#根据边edge，返回最小覆盖路径path
def return_path(df_jiedian,edge_path):
    G = nx.Graph()
    file=open(edge_path,'r')#以只读方式打开文件。文件的指针将会放在文件的开头。
    top_nodes=set()
    i=0
    for l in file:
        i=i+1
        l=l.rstrip('\n')
        l=l.split(' ')
        G.add_edges_from([l])
        top_nodes.add(l[0])
    #print('边数：',i)
    file.close()
    ######计算二分图中一个部分的点
    top_nodes=list(top_nodes)
    #print('顶点数：',len(top_nodes))
    #print(len(top_nodes))
    ######二分图计算最大匹配
    matching =nx.bipartite.eppstein_matching(G,top_nodes=top_nodes)#快
    ######去除最大匹配中的重复值
    new_match=[]
    for m in matching:
        if '^' in m:
            new_match.append((int(matching[m]),int(m[:-1])))
    #print('匹配数：',len(new_match))
    ######将匹配形成路径
    new_match=np.array(new_match)
    DiG = nx.DiGraph()
    DiG.add_edges_from(new_match)
    path_heji=list(nx.topological_sort(DiG))#path合在一起
    path=[]#存储覆盖路径
    q=0
    for i in range(len(path_heji)):
        if path_heji[i] not in new_match[:,0]:
            path.append(path_heji[q:i+1])
            q=i+1
    #print(len(path))
    #print(path)
    ######计算没有被匹配的点，单个节点为一条轨迹的节点
    dange=set(df_jiedian.index)-set(path_heji)#存储单个的节点
    for d in dange:
        path.append([d])
    #print('路径数：',len(path))
    return path

In [5]:
#1205 （8k 1w ，300 600 900） 
#1210 （8k 1w ，300 600 900）（100 120 140 160 180 200 220 240 260 280 300 320 340 360 380,8k） （900，1k-8k）
nn=1000
print('1210')
print('时间限制',900)
while nn<=10000:
    df_jiedian=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_jiedian.csv'),dtype= {'start_kid': str,'end_kid': str,'car_id': str})            
    df_kakou_time=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\kakou_time.csv'),dtype= {'origin_kid': str,'des_kid': str})
    df_kakou_time=df_kakou_time[df_kakou_time['time']<=900]#卡口时间限制900s
    print(nn,'节点')
    path_l=[]
    for p in range(10):
        slic=set()
        while len(slic)<nn:#随机取1w个节点
            slic.add(random.randint(0,len(df_jiedian)-1))
        slic=list(slic)
        df_jiedian1=df_jiedian.loc[slic]
        df_jiedian1['jiedian_id']=df_jiedian1.index
        save_edge(df_jiedian1,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_edge_test.txt')
        path=return_path(df_jiedian1,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_edge_test.txt')
        path_l.append(len(path))
        #print(len(path))
        #print(len(set(df_jiedian1['car_id'])))
        numpy_path = np.array(path,dtype=object)
        np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_path_test.npy',numpy_path )
        #print('车辆数：',len(set(df_jiedian1['car_id'])))
    print('路径数：',sum(path_l)//10)
    print('\n')
    nn=nn+200

1210
时间限制 900
1000 节点
路径数： 183


1200 节点
路径数： 204


1400 节点
路径数： 231


1600 节点
路径数： 252


1800 节点
路径数： 275


2000 节点
路径数： 304


2200 节点
路径数： 319


2400 节点
路径数： 342


2600 节点
路径数： 366


2800 节点
路径数： 383


3000 节点
路径数： 408


3200 节点
路径数： 440


3400 节点
路径数： 452


3600 节点
路径数： 476


3800 节点
路径数： 503


4000 节点
路径数： 531


4200 节点
路径数： 548


4400 节点
路径数： 568


4600 节点
路径数： 600


4800 节点
路径数： 610


5000 节点
路径数： 628


5200 节点
路径数： 652


5400 节点
路径数： 689


5600 节点
路径数： 695


5800 节点
路径数： 713


6000 节点
路径数： 736


6200 节点
路径数： 761


6400 节点
路径数： 781


6600 节点
路径数： 796


6800 节点
路径数： 836


7000 节点
路径数： 842


7200 节点
路径数： 865


7400 节点
路径数： 884


7600 节点
路径数： 903


7800 节点
路径数： 938


8000 节点
路径数： 960


8200 节点
路径数： 974


8400 节点
路径数： 995


8600 节点
路径数： 1029


8800 节点
路径数： 1018


9000 节点
路径数： 1060


9200 节点
路径数： 1080


9400 节点
路径数： 1094


9600 节点
路径数： 1126


9800 节点
路径数： 1154


10000 节点
路径数： 1162




In [5]:
'''
df_jiedian=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_jiedian.csv'),dtype= {'start_kid': str,'end_kid': str,'car_id': str})            
df_kakou_time=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\kakou_time.csv'),dtype= {'origin_kid': str,'des_kid': str})
df_kakou_time=df_kakou_time[df_kakou_time['time']<=900]#卡口时间限制900s


print(1)
df_jiedian1=df_jiedian[:100000]
df_jiedian1['jiedian_id']=df_jiedian1.index
save_edge(df_jiedian1,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_1.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_1.txt')
print(len(path))
print(len(set(df_jiedian1['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_1.npy',numpy_path )

print(2)
df_jiedian2=df_jiedian[100000:200000]
df_jiedian2['jiedian_id']=df_jiedian2.index
save_edge(df_jiedian2,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_2.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_2.txt')
print(len(path))
print(len(set(df_jiedian2['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_2.npy',numpy_path )

print(3)
df_jiedian3=df_jiedian[200000:300000]
df_jiedian3['jiedian_id']=df_jiedian3.index
save_edge(df_jiedian3,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_3.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_3.txt')
print(len(path))
print(len(set(df_jiedian3['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_3.npy',numpy_path )

print(4)
df_jiedian4=df_jiedian[300000:400000]
df_jiedian4['jiedian_id']=df_jiedian4.index
save_edge(df_jiedian4,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_4.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_4.txt')
print(len(path))
print(len(set(df_jiedian4['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_4.npy',numpy_path )

print(5)
df_jiedian5=df_jiedian[400000:500000]
df_jiedian5['jiedian_id']=df_jiedian5.index
save_edge(df_jiedian5,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_5.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_5.txt')
print(len(path))
print(len(set(df_jiedian5['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_5.npy',numpy_path )

print(6)
df_jiedian6=df_jiedian[500000:600000]
df_jiedian6['jiedian_id']=df_jiedian6.index
save_edge(df_jiedian6,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_6.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_6.txt')
print(len(path))
print(len(set(df_jiedian6['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_6.npy',numpy_path )

print(7)
df_jiedian7=df_jiedian[600000:700000]
df_jiedian7['jiedian_id']=df_jiedian7.index
save_edge(df_jiedian7,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_7.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_7.txt')
print(len(path))
print(len(set(df_jiedian7['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_7.npy',numpy_path )

print(8)
df_jiedian8=df_jiedian[700000:800000]
df_jiedian8['jiedian_id']=df_jiedian8.index
save_edge(df_jiedian8,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_8.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_8.txt')
print(len(path))
print(len(set(df_jiedian8['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_8.npy',numpy_path )

print(9)
df_jiedian9=df_jiedian[800000:900000]
df_jiedian9['jiedian_id']=df_jiedian9.index
save_edge(df_jiedian9,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_9.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_9.txt')
print(len(path))
print(len(set(df_jiedian9['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_9.npy',numpy_path )

print(10)
df_jiedian10=df_jiedian[900000:1000000]
df_jiedian10['jiedian_id']=df_jiedian10.index
save_edge(df_jiedian10,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_10.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_10.txt')
print(len(path))
print(len(set(df_jiedian10['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_10.npy',numpy_path )

print(11)
df_jiedian11=df_jiedian[1000000:1100000]
df_jiedian11['jiedian_id']=df_jiedian11.index
save_edge(df_jiedian11,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_11.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_11.txt')
print(len(path))
print(len(set(df_jiedian11['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_11.npy',numpy_path )

print(12)
df_jiedian12=df_jiedian[1100000:1200000]
df_jiedian12['jiedian_id']=df_jiedian12.index
save_edge(df_jiedian12,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_12.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_12.txt')
print(len(path))
print(len(set(df_jiedian12['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_12.npy',numpy_path )

print(13)
df_jiedian13=df_jiedian[1200000:]
df_jiedian13['jiedian_id']=df_jiedian13.index
save_edge(df_jiedian13,df_kakou_time,5,r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_13.txt')
path=return_path(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_edge_13.txt')
print(len(path))
print(len(set(df_jiedian13['car_id'])))
numpy_path = np.array(path)
np.save(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path_13.npy',numpy_path )
'''

#numpy_path=np.load(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_path.npy',allow_pickle=True)

"\nprint(1)\ndf_jiedian1=df_jiedian[:100000]\ndf_jiedian1['jiedian_id']=df_jiedian1.index\nsave_edge(df_jiedian1,df_kakou_time,5,r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181P5_edge_1.txt')\npath=return_path(r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181P5_edge_1.txt')\nprint(len(path))\nprint(len(set(df_jiedian1['car_id'])))\nnumpy_path = np.array(path)\nnp.save(r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181P5_path_1.npy',numpy_path )\n\nprint(2)\ndf_jiedian2=df_jiedian[100000:200000]\ndf_jiedian2['jiedian_id']=df_jiedian2.index\nsave_edge(df_jiedian2,df_kakou_time,5,r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181P5_edge_2.txt')\npath=return_path(r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181P5_edge_2.txt')\nprint(len(path))\nprint(len(set(df_jiedian2['car_id'])))\nnumpy_path = np.array(path)\nnp.save(r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181P5_path_2.npy',numpy_path )\n\nprint(3)\ndf_jiedian3=df_jiedian[200000:300000]\ndf_jiedian3['jiedian_id']=df_jiedian3.index\nsave_edge(df_jiedian3,df_kakou_time,5,r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181P5_edge_3.txt')

In [ ]:
'''
#5对应jiedian_id；7对应des_kid；9对应q_kakou_time
df_q_kakou=pd.merge(df_jiedian,df_kakou_time,left_on='end_kid',right_on='origin_kid',how='inner')
df_q_kakou['q_kakou_time']=df_q_kakou['time']+df_q_kakou['end_time']
#print(df_q_kakou[:5])
edge=[]
for i in range(len(df_q_kakou)):
    print(i)
    df_h_jiedian=df_jiedian[(df_jiedian['start_kid']==df_q_kakou.iloc[i,7]) & (df_jiedian['start_time']>df_q_kakou.iloc[i,9])]   
    df_h_jiedian_index=list(df_h_jiedian.index)
    for h_index in df_h_jiedian_index:
        edge.append((str(df_q_kakou.iloc[i,5]),str(h_index)+'^'))
'''

In [ ]:
''' 
#计算边的方法，慢
r_node_l=[]
for i in range(len(df_kakou_time)):
    df_kakou_h=df_jiedian[df_jiedian['start_kid']==df_kakou_time.iloc[i,1]]#1对应des_kid
    #print(df_kakou_time.iloc[i,1])
    #node_index_l=[str(node_index) for node_index in df_kakou_h.index]
    node_index_l=df_kakou_h.index
    r_node_l.append(node_index_l)
df_kakou_time['r_node_index']=r_node_l
#print(df_kakou_time)
df_q_kakou_h=pd.merge(df_jiedian,df_kakou_time,left_on='end_kid',right_on='origin_kid',how='inner')
df_q_kakou_h['q_time']=df_q_kakou_h['time']+df_q_kakou_h['end_time']

edge=[]
for i in range(len(df_q_kakou_h)):
    #9对应r_node_index
    #10对应q_time
    #5对应jiedian_id
    r_node=df_q_kakou_h.iloc[i,9]
    q_time=df_q_kakou_h.iloc[i,10]
    for n in r_node:
        if df_jiedian.iloc[n]['start_time']>q_time:     
            edge.append((str(df_q_kakou_h.iloc[i,5]),str(n)+'^'))
'''

In [ ]:
def return_edge(df_jiedian,df_kakou_time,time_restrict,start_kid_ord,end_kid_ord,start_time_ord,end_time_ord):   
    edge=[]
    df_kakou_time=df_kakou_time[df_kakou_time['time']<=time_restrict]
    df_kakou_time.reset_index(drop=True,inplace=True)#重置索引
    origin_kid_l=list(set(df_kakou_time['origin_kid']))
    #print(df_kakou_time)
    #print(len(origin_kid_l))
    for i in range(len(df_jiedian)):
        print(i)
        end_kid=df_jiedian.iloc[i,end_kid_ord]
        end_time=df_jiedian.iloc[i,end_time_ord]
        if end_kid in origin_kid_l:
            des_kid_l=list(set(df_kakou_time[df_kakou_time['origin_kid']==end_kid]['des_kid']))
            #根据des_kid到节点中找
            d=pd.DataFrame(des_kid_l,columns=['start_kid'])
            df_merge=pd.merge(df_jiedian,d,on='start_kid',how='inner')
            for j in range(len(df_merge)):
                time=df_merge.iloc[j,2]-end_time
                
        #else:
            #该节点不会连边
    '''
    edge=[]
    threads = []
    for i in range(len(df_jiedian)):
        print(i)
        for j in range(len(df_jiedian)):
            if i!=j:
                
                #time=df_kakou_time[(df_kakou_time['origin_kid']==df_jiedian.iloc[i,end_kid_ord])&(df_kakou_time['des_kid']==df_jiedian.iloc[j,start_kid_ord])]['time'].iloc[0]       
                #if (df_jiedian.iloc[i,end_time_ord]+time<=df_jiedian.iloc[j,start_time_ord])&(time<=time_restrict):
                    #添加边
                    #edge.append((str(i),str(j)+'^',time))
                
                h=MyThread(myfun,(df_jiedian,df_kakou_time,time_restrict,start_kid_ord,end_kid_ord,start_time_ord,end_time_ord,i,j),myfun.__name__)
                h.start()
                #print(h)
                threads.append(h)
    #print(threads)
    for t in threads:
        #print(t)
        t.join()# 一定要join，不然主线程比子线程跑的快，会拿不到结果
        edge.append(t.getResult())
    
    
    #删除列表中的None值
    new_edge = []
    for val in edge:
        if val != None :
            new_edge.append(val)
    #print(new_edge)
    df_edge=pd.DataFrame(new_edge,columns=['l','r','time'])
    return df_edge
    '''

In [ ]:
def return_G_matching(df_jiedian,df_kakou_time,edge):      
    G = nx.DiGraph()
    G.add_weighted_edges_from(edge)
    matching =nx.max_weight_matching(G)
    return G,matching

In [ ]:
time_restrict=900 #时间限制900s，时间要不要一起同除某个数
df_edge=return_edge(df_jiedian,df_kakou_time,time_restrict,start_kid_ord,end_kid_ord,start_time_ord,end_time_ord)        

In [ ]:
G,matching=return_G_matching(df_jiedian,df_kakou_time,edge)
#如何保存edge,G,matching
#最小路径覆盖数
path_minmum_number=len(df_jiedian)-len(matching)
#覆盖路径
new_jiedian=[]
for m in matching:
    for q in m:
        if '^' in q:
            end_kid=q
        else:
            start_kid=q
            
            
df_new_jiedian=pd.DataFrame(columns=['start_kid','end_kid','start_time','end_time']) 